In [104]:
import numpy as np
import tensorflow as tf
from utils_functions import *
from plan import *
import random
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from sklearn.model_selection import train_test_split


In [5]:

dict_stati= load_file("C:/Users/Francesco/Desktop/dataset/dataset/dizionario_stati")
plansLoaded=load_file("C:/Users/Francesco/Desktop/dataset/dataset/plans")
#print(type(plansPickles))
#plansList=load_from_pickles("C:/Users/Francesco/Desktop/dataset/dataset/dizionario_stati",plansPickles)
#print(plansPickles)


File loaded
File loaded


In [68]:
#ogni stato è un tensore con elementi dtype int8, questi stati vengono raggruppati in un altro tensore che rappresnta la
#variabile "states" del piano 
def build_vector(dict, states_list):
    l=len(dict)
    vector_states=[]
    for state in states_list:
        vector=np.array([0]*l,dtype=np.int8)
        #vector=tf.zeros(l,dtype=np.int8)
        for s in state:
            for key in dict.keys():
                if key==s:
                    vector[dict[key]-1]=1
                    break
        t=tf.convert_to_tensor(vector,dtype=tf.int8)            
        vector_states.append(t) 
    r = tf.convert_to_tensor(vector_states,dtype=None)                          
    return r


#NON UTILIZZATO ADESSO
#shape di ogni singolo elemento (r) è (n x 340) con n che varia su ogni piano, raggruppati sulla base dei piani
def build_all_vectors(dict,plans_list):
    total=[]
    for plan in plans_list:
        r=build_vector(dict,plan.states)
        total.append(r)
    #r = tf.convert_to_tensor(total,dtype=None)                         
    return total
 
    
#gli stati vengono ordinati (in una lista) singolarmente con shape (1x340), non vengono raggruppati sulla base dei piani
#da utilizzare per autoencoder standard
def build_all_vectors1x340(dict, plans_list):
    l=len(dict)
    total=[]
    for plan in plans_list:
        for state in plan.states:
            vector=np.array([0]*l,dtype=np.int8)
            for s in state:
                for key in dict.keys():
                    if key==s:
                        vector[dict[key]-1]=1
                        break
            t=tf.convert_to_tensor(vector,dtype=tf.int8)            
            total.append(t)
    return total        

In [47]:
#all_states_from_plans_list=build_all_vectors(dict_stati,plansLoaded)

## Preparazione dati

In [63]:
all1x340=build_all_vectors1x340(dict_stati,plansLoaded)
random.shuffle(all1x340)

In [122]:
#creAzione di train, test e validation set,--> input shape per rete = 340
#rendere questi set statici es. comprimerli 
#random.shuffle(all1x340)
half=int(len(all1x340)//3)
train=all1x340[half:]
vt=all1x340[:half]
h2=int(len(vt)//2)
validation=vt[h2:]
test=vt[:h2]


# Print di test

In [124]:
print(validation[59])
print(test[2000])
print(train[5000])

tf.Tensor(
[0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0
 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0], shape=(340,), dtype=int8)
tf.Tensor(
[1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [48]:
for i in range(3):
    print(all_states_from_plans_list[i])

tf.Tensor(
[[1 1 1 ... 0 0 0]
 [1 1 1 ... 0 0 0]
 [1 0 1 ... 0 0 0]
 ...
 [0 1 1 ... 0 0 0]
 [0 1 1 ... 0 0 0]
 [0 1 1 ... 0 0 0]], shape=(12, 340), dtype=int8)
tf.Tensor(
[[1 1 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 [1 1 0 ... 0 0 0]
 ...
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]], shape=(11, 340), dtype=int8)
tf.Tensor(
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]
 [1 0 0 ... 0 0 0]], shape=(15, 340), dtype=int8)


In [4]:
print("********initial state *******")
print(len(plansLoaded[1].initial_state))
print("")
print(plansLoaded[1].initial_state)

print("******** states *******")
print(len(plansLoaded[1].states))
print("")
print(plansLoaded[1].states)
print("******* goals ********")
print(len(plansLoaded[1].goals))
print("")
print(plansLoaded[1].goals)

print("\n\n -------------  dict stati -----------")
print(len(dict_stati))
print("")
print(dict_stati)

********initial state *******
14

['clear crate3', 'clear pallet0', 'at crate0 distributor0', 'available hoist4', 'at truck0 distributor0', 'available hoist3', 'available hoist2', 'on crate3 pallet4', 'at crate3 depot1', 'on crate0 pallet1', 'available hoist0', 'clear crate0', 'clear pallet2', 'at truck2 distributor2']
******** states *******
11

[['at truck2 depot1', 'clear crate0', 'available hoist0', 'on crate0 pallet1', 'clear crate3', 'on crate3 pallet4', 'available hoist4', 'at crate0 distributor0', 'available hoist2', 'available hoist3', 'at crate3 depot1', 'clear pallet0', 'clear pallet2', 'at truck0 distributor0'], ['at truck2 depot1', 'lifting hoist2 crate0', 'clear pallet1', 'available hoist0', 'clear crate3', 'on crate3 pallet4', 'available hoist4', 'available hoist3', 'at crate3 depot1', 'clear pallet0', 'clear pallet2', 'at truck0 distributor0'], ['at truck2 depot1', 'clear pallet1', 'in crate0 truck0', 'available hoist0', 'clear crate3', 'on crate3 pallet4', 'available h

In [64]:
for i in range(5):
    print(all1x340[i])

tf.Tensor(
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0], shape=(340,), dtype=int8)
tf.Tensor(
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 